In [1]:
import pandas as pd
import json
pd.options.display.max_rows = 10000
from pandas.io.json import json_normalize

In [2]:
#import jan 2018 - oct 2018 citi bike trip data
trips = pd.read_csv('tripdata_2018.csv')

In [3]:
#change data and station number data types
trips['starttime'] = pd.to_datetime(trips['starttime'])
trips['stoptime'] = pd.to_datetime(trips['stoptime'])
trips['start station id'] = trips['start station id'].fillna(0).astype(str)
trips['end station id'] = trips['end station id'].fillna(0).astype(str)

In [4]:
trips.head()

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,W 52 St & 11 Ave,40.767272,-73.993929,505.0,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1
1,1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,W 52 St & 11 Ave,40.767272,-73.993929,3255.0,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1
2,2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,W 52 St & 11 Ave,40.767272,-73.993929,525.0,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1
3,3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,W 52 St & 11 Ave,40.767272,-73.993929,447.0,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1
4,4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,W 52 St & 11 Ave,40.767272,-73.993929,3356.0,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1


In [5]:
#get list of possible e-bikes

In [239]:
#citi bike launched e-bikes on august 20, 2018
#get trips that occured from launch day to the end of august or after launch (al)
trips_al = trips[trips['starttime'] >= '2018-08-20']
trips_al = trips_al[trips_al['starttime'] <= '2018-08-31']

#remove duplicates to isolate distince bikes that existed after launch
bikeids_al = trips_al[['bikeid']].drop_duplicates(keep='first')
bikeids_al.head()

,bikeid
10700351,30808
10700352,21382
10700353,20847
10700354,28424
10700355,33372


In [240]:
#get trips that occured before launch (bl)
trips_bl = trips[trips['starttime'] < '2018-08-20']

#remove duplicates to isolate distince bikes that existed after launch
bikeids_bl = trips_bl[['bikeid']].drop_duplicates(keep='first')

In [241]:
#right join dataframes where the bikes that exist after launch but don't exist before are null 
bikeids_al['after'] = 1
bikeids_bl['before'] = 1
bikeid_merge = pd.merge(bikeids_bl, bikeids_al, on='bikeid', how='right')

#keep null bikes as possible ebikes
possible_ebikes = bikeid_merge[bikeid_merge.before.isnull()][['bikeid']].sort_values(by=['bikeid'])
possible_ebikes['bikeid'] = pd.to_numeric(possible_ebikes['bikeid'])
possible_ebikes.head()

,bikeid
10779,17635
10766,17672
10759,18333
10697,19254
10686,24812


In [9]:
#count number of possible ebikes
possible_ebikes.count()

bikeid    231
dtype: int64

In [10]:
#save possible_ebikes to csv
possible_ebikes.to_csv('potential_ebikes.csv')

In [23]:
#from trip df, get times when bikes were docked at stations

In [27]:
#get trip data for when station snapshots were taken as snapshot timeframe (ss_tf)
ss_tf = trips[trips['stoptime'] >= '2018-10-15']
ss_tf = ss_tf[ss_tf['stoptime'] < '2018-11-01']
ss_tf.count()

#sort df by 'bikeid' and 'starttime'
ss_tf = ss_tf.sort_values(['bikeid','starttime'])

In [29]:
#from trip data (start station A to end station B), get time at dock (end station B to start station B)
#to do this, in sorted df shift 'starttime', 'start station id' and 'bikeid' one row up
shifted_stations = ss_tf[['starttime']].shift(-1)
shifted_stations['bikeid_shift'] = ss_tf[['bikeid']].shift(-1)
shifted_stations['start_station_id_shift'] = ss_tf[['start station id']].shift(-1)
 
#rename 'starttime' column that was shifted
shifted_stations = shifted_stations.rename(columns={"starttime":"starttime_shift"})

In [33]:
#join snapshot timeframe df with shifted stations to get time at dock (end station B from ss_tf to start station B from shifter_stations)
docked_bikes = pd.merge(ss_tf, shifted_stations, left_index=True, right_index=True, how='left')
docked_bikes = docked_bikes.sort_values(['bikeid','starttime'])

In [34]:
#with sorted df, remove the last trip of each bike(when bikeid != bikeid)
#remove rows where 'end station id' != 'start_station_id_shift' 
docked_bikes = docked_bikes[docked_bikes['bikeid'] == docked_bikes['bikeid_shift']]
docked_bikes = docked_bikes[docked_bikes['end station id'] == docked_bikes['start_station_id_shift']]
docked_bikes.count()

Unnamed: 0                 925241
tripduration               925241
starttime                  925241
stoptime                   925241
start station id           925241
start station name         925194
start station latitude     925241
start station longitude    925241
end station id             925241
end station name           925194
end station latitude       925241
end station longitude      925241
bikeid                     925241
usertype                   925241
birth year                 925241
gender                     925241
starttime_shift            925241
bikeid_shift               925241
start_station_id_shift     925241
dtype: int64

In [35]:
docked_bikes.head()

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,starttime_shift,bikeid_shift,start_station_id_shift
14309326,916504,354,2018-10-15 07:14:31.684,2018-10-15 07:20:26.594,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,168.0,W 18 St & 6 Ave,40.739713,-73.994564,14529,Subscriber,1993,1,2018-10-15 07:54:53.735,14529.0,168.0
14311583,918761,607,2018-10-15 07:54:53.735,2018-10-15 08:05:01.548,168.0,W 18 St & 6 Ave,40.739713,-73.994564,524.0,W 43 St & 6 Ave,40.755273,-73.983169,14529,Subscriber,1995,1,2018-10-15 16:46:20.623,14529.0,524.0
14338718,945896,484,2018-10-15 16:46:20.623,2018-10-15 16:54:25.333,524.0,W 43 St & 6 Ave,40.755273,-73.983169,495.0,W 47 St & 10 Ave,40.762699,-73.993012,14529,Subscriber,1961,1,2018-10-15 16:59:37.475,14529.0,495.0
14339756,946934,782,2018-10-15 16:59:37.475,2018-10-15 17:12:39.793,495.0,W 47 St & 10 Ave,40.762699,-73.993012,440.0,E 45 St & 3 Ave,40.752554,-73.972826,14529,Subscriber,1958,1,2018-10-15 17:24:56.899,14529.0,440.0
14342488,949666,795,2018-10-15 17:24:56.899,2018-10-15 17:38:12.146,440.0,E 45 St & 3 Ave,40.752554,-73.972826,492.0,W 33 St & 7 Ave,40.750200,-73.990931,14529,Subscriber,1978,1,2018-10-15 18:19:14.382,14529.0,492.0


In [36]:
docked_bikes.to_csv('bikes_at_station.csv')

In [180]:
#get station snapshots to see which stations had ebikes at certain times
#station snapshots were taken from citi bike's live station information feed at random intervals during october

In [242]:
#create livefeed (lf) dictionary
lf_dict = {'2018-10-16 10:42:00': '2018-10-16_104200', '2018-10-16 10:52:00': '2018-10-16_105200', 
'2018-10-17 19:39:00': '2018-10-17_193900', '2018-10-21 11:30:00': '2018-10-21_113000', 
'2018-10-25 23:33:00': '2018-10-25_233300', '2018-10-26 13:52:00': '2018-10-26_135200', 
'2018-10-29 12:25:00': '2018-10-29_122500', '2018-10-29 19:03:00': '2018-10-29_190300',
'2018-10-30 20:24:00': '2018-10-30_202400', '2018-10-30 22:48:00': '2018-10-30_224800'}

In [258]:
#gets the bikes from 'poss_ebikes' that are false positives (ie not e-bikes)

disqualified_bikes = pd.DataFrame()
for key in lf_dict:

    other_date_format = lf_dict[key]
    lf_json = pd.read_json('live_feed/citibike_live_feed_' + other_date_format + '.json')
    lf = json_normalize(lf_json['data'][0])

    #remove false positive ebikes
    #get stations without any ebikes
    lf = lf[lf['num_ebikes_available'] == 0]

    #get bikes that were at stations when snapshot was taken
    docked = docked_bikes[docked_bikes['starttime_shift'] >= key]
    docked = docked[docked['stoptime'] <= key]

    #get the bikeids present in 'possible_ebikes' and 'disqualified'
    disqualify = docked.merge(possible_ebikes, how='inner', on=['bikeid','bikeid'])
    disqualified_bikes = disqualified_bikes.append(disqualify[['bikeid']])

In [259]:
#remove duplicates
disqualified_bikes = disqualified_bikes[['bikeid']].drop_duplicates(keep='first')

In [260]:
#get the possible ebike again and remove the disqualified bikes
disqualified_bikes['type'] = 1
poss_ebikes_2 = possible_ebikes.merge(disqualified_bikes, how='left', on='bikeid')
poss_ebikes_2 = poss_ebikes_2[poss_ebikes_2['type'].isnull()]
poss_ebikes_2.head()

,bikeid,type
4,24812,NaN
5,26194,NaN
10,29289,NaN
14,34422,NaN
15,34425,NaN


In [261]:
#add type of bike identifier to trips df (1 = possible ebike, 0 = regular bike)
poss_ebikes_2 = poss_ebikes_2.fillna(1)
trips = trips.merge(disqualified_bikes[['bikeid','type']], how='left', on='bikeid').fillna(0)

In [272]:
trips.to_csv('trips_ebike_id.csv')

In [267]:
#try out the same exercise using all bike ids (without possible ebike)

bike_list = trips[['bikeid']].drop_duplicates(keep='first')
bike_list.count()

bikeid    14851
dtype: int64

In [268]:
#gets the bikes from 'poss_ebikes' that are false positives (ie not e-bikes)

disqualified_bikes = pd.DataFrame()
for key in lf_dict:

    other_date_format = lf_dict[key]
    lf_json = pd.read_json('live_feed/citibike_live_feed_' + other_date_format + '.json')
    lf = json_normalize(lf_json['data'][0])

    #remove false positive ebikes
    #get stations without any ebikes
    lf = lf[lf['num_ebikes_available'] == 0]

    #get bikes that were at stations when snapshot was taken
    docked = docked_bikes[docked_bikes['starttime_shift'] >= key]
    docked = docked[docked['stoptime'] <= key]

    #get the bikeids present in 'possible_ebikes' and 'disqualified'
    disqualify_all = docked.merge(bike_list, how='inner', on=['bikeid','bikeid'])
    disqualified_bikes_all = disqualified_bikes_all.append(disqualify_all[['bikeid']])

/anaconda2/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [269]:
#remove duplicates
disqualified_bikes_all = disqualified_bikes_all[['bikeid']].drop_duplicates(keep='first')
disqualified_bikes_all.count()

bikeid    9685
dtype: int64

In [270]:
#get the possible ebike list without the disqualified bikes
disqualified_bikes_all['value_disq'] = 1
poss_ebikes_all = bike_list.merge(disqualified_bikes_all, how='left', on='bikeid')
poss_ebikes_all = poss_ebikes_all[poss_ebikes_all['value_disq'].isnull()]
poss_ebikes_all.count()

bikeid        5166
value_disq       0
dtype: int64